# Make tables: calculate p-values and differential expressions for each cancer 

Create a dataframe with p-value results from t-tests for all proteins (trans proteomics when PTEN has cnv deletions compared to PTEN wildtype). The dataframe also includes the change in medians between deletions and wildtype. Prepare these tables for further analysis by creating csv files.

In [1]:
import pandas as pd
import numpy as np
import re

import cptac
import cptac.pancan as pc
import cptac.utils as u
import plot_utils as p

In [2]:
print('cptac version:', cptac.version())

cptac version: 1.0.0


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
'''
Params
cancer_object: Object. The loaded cancer data set.
all_prot: List. trans proteins used to get proteomics data. 
gene_in: String. Gene used to get mutation status for the Mutation col.
utils: utils package from cptac.  

Returns a dataframe with trans proteomics and mutation status of a specific gene.
'''

def all_prot_format_df(cancer_object, all_prot, gene_in = 'PTEN', utils = u):
    mut_type = cancer_object.get_genotype_all_vars(gene_in, omics_source = 'washu') # washu has cnv
    if cancer_object.get_cancer_type() not in ('pancanluad'):
        # Keep only tumor samples from proteomics
        prot_and_mutations = cancer_object.join_omics_to_mutations(mutations_genes = [gene_in], 
                                            omics_df_name = 'proteomics', omics_source = 'umich', 
                                            omics_genes = all_prot, tissue_type = 'tumor')
        # Reduce a multiindex 
        if isinstance(prot_and_mutations.keys(), pd.core.indexes.multi.MultiIndex):
            prot_and_mutations = utils.reduce_multiindex(prot_and_mutations, levels_to_drop = 1)
            prot_and_mutations = rename_duplicate_cols(prot_and_mutations) # add '_i' to distinguish
            
        # Merge Mutation column from get_genotype_all_vars (includes cnv) with proteomics
        mut_type = mut_type[['Mutation']] # get mutation column (created in get_genotype_all_vars)
        prot_df = prot_and_mutations.iloc[:,:-4] # Keep only proteomics
        merged = prot_df.join(mut_type)
        
        # Keep only Wildtype and deletion
        compare = ['Wildtype_Tumor','Deletion']
        get = merged['Mutation'].isin(compare)
        mut_wt = merged[get]
    
    
    # Luad has no somatic mutations for PTEN which changes some things
    elif cancer_object.get_cancer_type() in ('pancanluad'): 
        # get_genotype_all_vars add cnv data under the column PTEN
        mut_type = mut_type.drop(columns= gene_in)
        # different code because no somatic mutation data for pten (can't join to somatic mutations)
        omics = cancer_object.join_omics_to_omics(df1_name = 'CNV', df2_name='proteomics', df1_source = 'washu', 
                                                  df2_source = 'umich', genes1= gene_in, genes2= all_prot, 
                                                  tissue_type = 'tumor')
        omics = utils.reduce_multiindex(omics, levels_to_drop = 1, flatten = True)
        omics = rename_duplicate_cols(omics)
        omics = omics.drop(columns='PTEN_washu_CNV')
       
        # Merge Mutation column from get_genotype_all_vars (includes cnv) with proteomics
        merged = omics.join(mut_type) 
        # Keep only No_Mutation (wildtype) and deletion
        compare = ['No_Mutation','Deletion']
        get = merged['Mutation'].isin(compare)
        mut_wt = merged[get]
        mut_wt['Mutation'] = np.where(
            mut_wt['Mutation'] == 'No_Mutation', 'Wildtype_Tumor', 'Deletion')
    
    mut_wt = mut_wt.rename(columns={col: re.sub(r'_umich_proteomics', '', col) for col in mut_wt.columns.tolist()})
    return mut_wt


In [5]:
'''
Params
cancer_object: Object of the loaded cancer data set.
cancer_name: String. Name to add to the created df.
del_wt_df: DataFrame. df of only samples with PTEN deletion and wildtype. 
all_prot_list: List. All proteins in proteomics df. 

Returns a df with the differential expression between proteomics with PTEN del and wt (del - wt). 
'''

def get_change_in_medians_df(cancer_object, cancer_name, del_wt_df, all_prot_list):
    if  cancer_object.get_cancer_type() in ('pancanucec'):
        d = del_wt_df[del_wt_df.Mutation == "Truncation"]
        wt = del_wt_df[del_wt_df.Mutation == "Wildtype_Tumor"]
    else:
        d = del_wt_df[del_wt_df.Mutation == "Deletion"]
        wt = del_wt_df[del_wt_df.Mutation == "Wildtype_Tumor"]
    del_med = d.median()
    wt_med = wt.median()

    med_dict = {}
    
    # Correlation: + is mutant up compared to wt, - is mutant down
    for prot in all_prot_list:
        if prot in del_med.index and prot in wt_med.index:
            dif = del_med[prot] - wt_med[prot]
            med_dict[prot] = dif
        
    df = pd.DataFrame.from_dict(med_dict, orient='index', columns=[cancer_name+'_Median'])
    df = df.reset_index().rename(columns={'index':'Umich_Proteomics'})
    
    return df
        

In [6]:
''' Adds '_i' to duplicate col names where the int i increases by one with each duplicate 
Returns a df with unique column names. '''

def rename_duplicate_cols(df):
    cols = pd.Series(df.columns[:])

    for dup in cols[cols.duplicated()].unique(): 
        cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]

    # rename the columns with the cols list.
    df.columns=cols
    return df

#  Step 1: Create data frames with p-values and differential expressions

Each cancer needs a data frame containing only samples that have PTEN cnv deletions and PTEN wildtype with trans proteomics. Use wrap_ttest to run t-tests for all genes in the proteomic data frame. Use get_change_in_medians_df to create the data frame with differential expression values. Merge both data frames. 

Load in cancer data sets from cptac. 

In [7]:
en = pc.PancanUcec()
h = pc.PancanHnscc()
l = pc.PancanLuad()
ls = pc.PancanLscc()
o = pc.PancanOv()
r = pc.PancanCcrcc()
co = pc.PancanCoad()
g = pc.PancanGbm()
b = pc.PancanBrca()

In [8]:
#pc.download('all', redownload = True)

# Gbm

Part 1: Format data frame using all_prot_format_df. This returns a dataframe with proteomics and a Mutation column containing only PTEN cnv deletions and wildtype tumors. 

In [9]:
gene = 'PTEN'
g_prot = g.get_proteomics(source = 'umich')
g_prot = g_prot.drop(columns = 'PTEN') # cis effect
g_prot_list = list(g_prot.columns.get_level_values(0))

g_del_wt = all_prot_format_df(g, g_prot_list)
g_del_wt.head()
g_del_wt.Mutation.value_counts()

Deletion          79
Wildtype_Tumor    17
Name: Mutation, dtype: int64

Part 2: Run T-tests. Create a data frame of just significant comparisons and another data frame for all comparisons. A moderately stringent correction is used (FDR_BH).

In [10]:
g_cols = list(g_del_wt.columns[:-1])

# Get all pvals
g_pval = u.wrap_ttest(g_del_wt, 'Mutation', g_cols, return_all = True, correction_method = 'fdr_bh', mincount = 5)
g_pval = g_pval.rename(columns = {'Comparison': 'Umich_Proteomics','P_Value': 'GBM_P_Value'})

In [11]:
g_pval

,Umich_Proteomics,GBM_P_Value
0,CUL2,4.572167e-07
1,DOCK1_1,4.572167e-07
2,PIP4K2A,2.291619e-06
3,CUTC,2.291619e-06
4,ATE1_1,2.291619e-06
...,...,...
11568,THAP10,9.996790e-01
11569,ZNF639,9.998768e-01
11570,ALDOB,9.999004e-01
11571,GPR37L1,9.999004e-01


Part 3: Create the differential expression df. (median of tumors with PTEN wildtype - median of tumors with PTEN cnv deletions)

In [12]:
g_cols = list(g_del_wt[:-1])
g_med = get_change_in_medians_df(g, "GBM", g_del_wt, g_cols)
g_med

,Umich_Proteomics,GBM_Median
0,A1BG,-0.280406
1,A2M,-0.550903
2,AAAS,0.112206
3,AACS,0.104499
4,AADAT,-0.375602
...,...,...
12996,ZWINT,-0.023144
12997,ZXDC,0.019906
12998,ZYG11B,0.108842
12999,ZYX,0.081073


Part 4: Merge the p-value and differential expression dfs.

In [13]:
g_merged = g_pval.merge(g_med, on='Umich_Proteomics', how='outer')
g_merged = g_merged.replace(to_replace = r'_proteomics', value = '', regex = True)

# Create csv
g_merged.to_csv('csv/Single_Cancer/GBM_pval_medians_pancan.csv',index=False)
g_merged.head()

,Umich_Proteomics,GBM_P_Value,GBM_Median
0,CUL2,4.572167e-07,-0.458427
1,DOCK1_1,4.572167e-07,-0.358361
2,PIP4K2A,2.291619e-06,-0.494985
3,CUTC,2.291619e-06,-0.490992
4,ATE1_1,2.291619e-06,-0.481942


# Repeat for other cancers.

In [14]:
cancer_objects = {'HNSCC':h, 'LUAD':l, 'LSCC':ls, 'BR':b, 'CO':co, 'OV':o}

In [15]:
import warnings
warnings.filterwarnings('ignore')

gene = 'PTEN'
merged_dfs = {}
for cancer in cancer_objects:
    print(cancer)
    prot = cancer_objects[cancer].get_proteomics(source = 'umich')
    prot = prot.drop(columns = 'PTEN') # cis effect
    
    # Reduce a multiindex 
    if isinstance(prot.keys(), pd.core.indexes.multi.MultiIndex):
        prot = u.reduce_multiindex(prot, levels_to_drop = 1)
    prot_list = list(prot.columns)
    
    # Format df for t-test
    del_wt = all_prot_format_df(cancer_objects[cancer], prot_list)
    
    # Get list of unique cols for t-test and get_change_in_medians
    prot_and_iso = list(del_wt.columns[:-1])

    # Get all pvals
    min_num = 5
    all_pval = u.wrap_ttest(del_wt, 'Mutation', prot_and_iso, return_all = True, 
                            correction_method = 'fdr_bh', mincount = min_num)
    all_pval = all_pval.rename(columns = {'Comparison': 'Umich_Proteomics','P_Value': cancer+'_P_Value'})
    
    # Get change in medians
    delta_median_df = get_change_in_medians_df(cancer_objects[cancer], cancer, del_wt, prot_and_iso)

    # Merge pval_df and delta_median_df
    pval_medians_df = all_pval.merge(delta_median_df, on='Umich_Proteomics', how='outer')
    merged_dfs[cancer] = pval_medians_df # testing purposes
    
    # Create csv
    pval_medians_df.to_csv('csv/Single_Cancer/'+cancer+'_pval_medians_pancan.csv', index=False)
    print(pval_medians_df.head(), '\n')

HNSCC
  Umich_Proteomics  HNSCC_P_Value  HNSCC_Median
0             CUL1       0.002757      0.292084
1             GCLC       0.008152      1.135814
2             ADI1       0.013467      0.462333
3           AKR1C3       0.013467      0.903635
4            NCBP1       0.013467      0.430034 

LUAD
  Umich_Proteomics  LUAD_P_Value  LUAD_Median
0           LANCL2      0.000865     0.386394
1             EGFR      0.001945     0.598962
2             WAPL      0.008763    -0.183739
3           ATE1_1      0.012016    -0.315236
4              STS      0.015439     0.877910 

LSCC
  Umich_Proteomics  LSCC_P_Value  LSCC_Median
0         ABRAXAS2      0.000098    -0.194459
1          CCDC186      0.000172    -0.303387
2          DENND10      0.000238    -0.190829
3             GBF1      0.000238    -0.219597
4            SLK_1      0.000238    -0.295107 

BR
  Umich_Proteomics  BR_P_Value  BR_Median
0           SPTBN2    0.736793   0.866577
1             MT2A    0.736793   1.477416
2        

# Endometrial

The Endometrial data set does not have enough cnv deletions to perform a t-test, however the data set does have enough truncation type mutations (nonsense and frameshift). Different code is needed to create the data frame for Endometrial.

In [16]:
gene = 'PTEN'
prot = en.get_proteomics(source = 'umich')
prot = u.reduce_multiindex(prot, levels_to_drop = 1)
prot = prot.drop(columns = 'PTEN')
e_prot_list = list(prot.columns)

# Get mutations based on priority filter when a sample has multiple mutations
mut_type = en.get_genotype_all_vars(gene, omics_source = 'washu')

# merge cnv with genotype all mut type
prot_and_mutations = en.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_source='umich', omics_genes = e_prot_list, tissue_type = 'tumor') # drop Normal samples
prot_and_mutations = u.reduce_multiindex(prot_and_mutations, levels_to_drop = 1)

prot_df = prot_and_mutations.iloc[:,:-4] #drop Mutation and Location cols
mut_type = mut_type[['Mutation']] # Get Mutation col that includes CNV
merged = prot_df.join(mut_type) # merge 

# Create Truncation category and keep truncation and wt
compare = ['Wildtype_Tumor','Nonsense_Mutation','Frame_Shift_Ins','Frame_Shift_Del']
get = merged['Mutation'].isin(compare)
trunc_wt = merged[get]
trunc_wt['Mutation'] = np.where(
                trunc_wt['Mutation'] == 'Wildtype_Tumor', 'Wildtype_Tumor', 'Truncation')
trunc_wt = rename_duplicate_cols(trunc_wt)

In [17]:
e_cols = list(trunc_wt.columns[:-1])

# Get all pvals
e_pval = u.wrap_ttest(trunc_wt, 'Mutation', e_cols, return_all = True, correction_method = 'fdr_bh', mincount = 5)
e_pval = e_pval.rename(columns = {'Comparison': 'Umich_Proteomics','P_Value': 'EC_P_Value'})

Differential expression with adaption for trunctation mutations.

In [18]:
t = trunc_wt[trunc_wt.Mutation == "Truncation"]
wt = trunc_wt[trunc_wt.Mutation == "Wildtype_Tumor"]
trunc_med = t.median()
wt_med = wt.median()

en_d = {}

for prot in e_cols:
    if prot in trunc_med.index and prot in wt_med.index:
        dif = trunc_med[prot] - wt_med[prot]
        en_d[prot] = dif

en_med = pd.DataFrame.from_dict(en_d, orient='index', columns=['EC_Median'])
en_med = en_med.reset_index().rename(columns={'index':'Umich_Proteomics'})

In [19]:
e_merged = e_pval.merge(en_med, on='Umich_Proteomics',how='outer')
e_merged = e_merged.replace(to_replace = r'_umich_proteomics', value = '', regex = True)

# Create csv
e_merged.to_csv('csv/Single_Cancer/EC_pval_medians_pancan.csv',index=False)
e_merged.head()

,Umich_Proteomics,EC_P_Value,EC_Median
0,ABT1,0.000731,-0.479300
1,NOL10,0.000731,-0.366391
2,UTP25,0.000912,-0.495638
3,ZC3HC1,0.000912,-0.184747
4,TJP3,0.000912,0.531499


# Get number of significant genes in each cancer

In [20]:
# Add Gbm and En to merged_dfs dictionary
merged_dfs['GBM'] = g_merged
merged_dfs['EC'] = e_merged

In [21]:
# Using corrected pvals from wrap_ttest, each cancer's cutoff is 0.05
all_sig = []

for cancer in merged_dfs:
    df = merged_dfs[cancer]
    sig_df = df.loc[df[cancer+'_P_Value'] < 0.05]
    print(cancer, 'sig comparisons:', len(sig_df))
    sig_list = list(sig_df['Umich_Proteomics'])
    all_sig.append(sig_list)
        

flat_list = [item for sublist in all_sig for item in sublist] #change list of lists, to just one list
sig = list(set(flat_list)) # keep only one if gene in multiple cancer sig lists
print('\nNumber of significant comparisons in at least 1 cancer:', len(sig))

HNSCC sig comparisons: 43
LUAD sig comparisons: 17
LSCC sig comparisons: 1146
BR sig comparisons: 0
CO sig comparisons: 11
OV sig comparisons: 0
GBM sig comparisons: 603
EC sig comparisons: 589

Number of significant comparisons in at least 1 cancer: 2147


# Get a list of significant genes in at least one cancer

In [22]:
s = pd.Series(sig)
print('Number of significant genes in at least one cancer:', len(s))
s.to_csv('csv/list_sig_one_cancer_pancan.csv', index=False)

Number of significant genes in at least one cancer: 2147


# Get a list of significant genes in multiple cancers

In [23]:
# sig in multiple cancers
mult = list(set(i for i in flat_list if flat_list.count(i) > 1)) # Keep genes sig in more than 1 cancer
print('Number of significant genes in mult cancers:', len(mult))

Number of significant genes in mult cancers: 240


In [24]:
m = pd.Series(mult)
m = m.replace(to_replace = r'_proteomics$', value = '', regex = True)
m.to_csv('csv/list_sig_multiple_cancers_pancan.csv', index=False)